1. Some shows do not exist on IMDB
2. Some shows do not exist on IMDB but the function grabs the top hit, grabbing the wrong rating
3. Some shows have wrong release years, Netflix uses last season's release year while IMDB uses initial release 
4. Some shows have matching names

1. The grab_ID function seems redundant and can be optimized greatly
*NOTE: Some of the redundancy is deliberate to allow room for a 'confidence' score. This can be an indicator of how sure the function was about the title_ID it grabbed.*
2. Addition of the confidence score.
3. Addition of a check_accuracy function which compares grabbed (IMDB) title names with the netflix_data title names and removes bad entries.
4. Addition of a check_ID_present function which reads the output of a previous run and uses that data for title_ID's instead of performing a full run. 
5. Addition of an update function. The function will only go through using the previous title_ID's by calling check_ID_present and just update the ratings as they change on IMDB frequently.
6. Addition of a fill_missing function. This will go through a previous run output by calling check_ID_present and only attempt to search for missing entries' title_ID's on IMDB.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
#Web Access and parsing
import urllib.request, urllib.parse
from bs4 import BeautifulSoup

# Importing Data
netflix_data = pd.read_csv("C:/Users/oisin/Downloads/archive (1)/netflix_titles.csv")
netflix_data.head() #Displaying first few entries
#np.where(pd.isnull(netflix_data.release_year)) #checking for empty values for release years, None found, **might implement later just in case**

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


In [3]:
#IMDB Access/Search and Parse HTML to grab title_ID
def grab_ID(title,year,media_type):
    encoding = 'utf-8' #standard internet encoding
    find_url = 'https://www.imdb.com/find' #base search URL for IMDB
    values = {'q':title} # What we are searching for
    query = urllib.parse.urlencode(values).encode(encoding) #parsing the query and encoding correctly for web to understand it
    req = urllib.request.Request(find_url,query) #request: the "link" you can paste in the search bar
    resp = urllib.request.urlopen(req) #sending the request and recording the response
    html = BeautifulSoup(resp,"html") #using Beautiful Soup 4 to process the HTML response
    #Finding top 3 results through <td> tags with result text class in the HTML response. <td> is a table cell in HTML
    search = html.findAll("td", {"class": "result_text"})[:3]
    if len(search) > 0:
        for item in search: 
            if  str(item.text).find(str(year)) > 0:
                if str(media_type) == "TV Show":
                    if str(item.text).find("Series") > 0:
                        title_names = str(item.text)
                        title_IDs = str(item.a["href"])
                        #confidence = 
                        break
                    else:
                        #confidence = 
                        title_names = str(item.text)
                        title_IDs = str(item.a["href"])
                        break
                else:
                    title_names = str(item.text)
                    title_IDs = str(item.a["href"])
                    #confidence =
                    break

            else:
                if str(media_type) == "TV Show":
                    if str(item.text).find("Series") > 0:
                        title_names = str(item.text)
                        title_IDs = str(item.a["href"])
                        #confidence =
                        break
                    else:
                        title_names = str(item.text)
                        title_IDs = str(item.a["href"])
                        #confidence =
                        break
                else:
                    title_names = "NaN"
                    title_IDs = "NaN"
                    #confidence =
                    break
    else:
        title_names = "NaM"
        title_IDs = "NaN"
        #confidence =

    return title_names, title_IDs

In [4]:
#Acessing IMDB and parse HTML to grab Ratings
def grab_Rating(title_ID):
    imdb_url = "https://www.imdb.com" #base imdb website
    if title_ID == "NaN":
        rating = "NaN"
    else:
        target_url = imdb_url + title_ID #url for title
        target_url = target_url
        resp = urllib.request.urlopen(target_url)
        html = BeautifulSoup(resp,"html") #using Beautiful Soup 4 to process the HTML responce
        search = html.findAll("div", {"class": "ratingValue"})
        if len(search) > 0:
            for item in search: #Grabing rating text
                rating = str(item.strong['title'])
        else:
            rating = "NaN"

    return rating

In [5]:
IMDB_rating = []
IMDB_titleID = []
IMDB_title_name = []
NumShows = len(netflix_data.index)
for i in range(NumShows):
    title_name, title_ID = grab_ID(netflix_data.title[i],netflix_data.release_year[i],netflix_data.type[i])
    IMDB_titleID.append(title_ID)
    IMDB_title_name.append(title_name)
    rating = grab_Rating(title_ID)
    IMDB_rating.append(rating)
    if i%100 == 0:
        print("Progress is: ", i)
    #print(netflix_data.title[i], "\t\t" title_name,"\t\t" , i)

Progress is:  0
Progress is:  100
Progress is:  200
Progress is:  300
Progress is:  400
Progress is:  500
Progress is:  600
Progress is:  700
Progress is:  800
Progress is:  900
Progress is:  1000
Progress is:  1100
Progress is:  1200
Progress is:  1300
Progress is:  1400
Progress is:  1500
Progress is:  1600
Progress is:  1700
Progress is:  1800
Progress is:  1900
Progress is:  2000
Progress is:  2100
Progress is:  2200
Progress is:  2300
Progress is:  2400
Progress is:  2500
Progress is:  2600
Progress is:  2700
Progress is:  2800
Progress is:  2900
Progress is:  3000
Progress is:  3100
Progress is:  3200
Progress is:  3300
Progress is:  3400
Progress is:  3500
Progress is:  3600
Progress is:  3700
Progress is:  3800
Progress is:  3900
Progress is:  4000
Progress is:  4100
Progress is:  4200
Progress is:  4300
Progress is:  4400
Progress is:  4500
Progress is:  4600
Progress is:  4700
Progress is:  4800
Progress is:  4900
Progress is:  5000
Progress is:  5100
Progress is:  5200
Progr

In [6]:
#Writing outputs
outputIMDB = pd.DataFrame({'IMDB_titleID': IMDB_titleID, 'IMDB_rating': IMDB_rating, 'IMDB_title_name': IMDB_title_name})
output_name = 'IMDB_results_' + 'March-15-2021' + '.csv'
outputIMDB.head()
outputIMDB.to_csv(output_name, index=False)
print("Complete!")

Complete!


In [63]:
netflix_data
outputIMDB 
Missing_info = 0
netflix_data["IMDB_Rating"] = "NaN"
for i in range(len(netflix_data.title)):
    # Cleaning the netflix df titles, then searhcing for that title in the cleaned IMDB titles.
    # If exists then append the IMDB score to  the netflix df
    # Otherwise label it as NaN.
    # Some of the Netflix titles have non-ascii characters in them, these characters will be removed
    clean_title = netflix_data.title[i].encode('ascii', errors='ignore').decode()
    imdb_clean_title = outputIMDB.IMDB_title_name[i].encode('ascii', errors='ignore').decode()
    if clean_title in imdb_clean_title:
        netflix_data.title[i] = clean_title
        netflix_data.IMDB_Rating[i] = outputIMDB.IMDB_rating[i]
    else:
        Missing_info += 1

        
print(netflix_data.head(len(netflix_data.title)))
print(Missing_info)

<ipython-input-63-f83af2082b64>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  netflix_data.title[i] = clean_title
<ipython-input-63-f83af2082b64>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  netflix_data.IMDB_Rating[i] = outputIMDB.IMDB_rating[i]


9
Oct-01
Feb-09
22-Jul
15-Aug
'89
​​Kuch Bheege Alfaaz
​SAINT SEIYA: Knights of the Zodiac
#blackAF
#realityhigh
Òlòtūré
10,000 B.C.
100 Days Of Solitude
100 Things to do Before High School
100 Years: One Woman's Fight for Justice
12 ROUND GUN
13TH
13TH: A Conversation with Oprah Winfrey & Ava DuVernay
14 Cameras
20 Feet From Stardom
21 Sarfarosh: Saragarhi 1897
27, el club de los malditos
27: Gone Too Soon
37 Seconds
40 Sticks
45 rpm
6-5=2
7SEEDS
A 2nd Chance
A Beautiful Life
A Boy Name Flora A
A Christmas Special: Miraculous: Tales of Ladybug & Cat Noir
A Family Affair
A Holiday Engagement
A Home with A View
A Land Imagined
A Love Song for Latasha
A Love Story
A Mighty Team
A Mission in an Old Movie
A new Capitalism
A Second Chance
A Shaun the Sheep Movie: Farmageddon
A truthful Mother
A Twelve Year Night
A Week in Watts
A Year In Space
A Young Doctor's Notebook and Other Stories
Aşk Tesadüfleri Sever
Aajcha Divas Majha
Aashik Awara
ABCD: Any Body Can Dance
Abdullah, The Final Witnes

For Love or Money
For the Birds
For Your Own Good
Francesco De Carlo: Cose di Questo Mondo
Franco Escamilla: Bienvenido al mundo
Frank and Cindy
Frankenstein’s Monster’s Monster, Frankenstein
Freak Show
Freaks – You're One of Us
Free Rein: Valentine's Day
Friendship
Froning: The Fittest Man in History
Fronteras
Frozen Planet: The Epic Journey
FullMetal Alchemist
Furie
Futmalls.com
FYRE: The Greatest Party That Never Happened
Gabriel lglesias: I’m Sorry For What I Said When I Was Hungry
Gaddar: the Traitor
Game
GAME ON: A Comedy Crossover Event
Game Over
Game Over (Hindi Version)
Game Over (Tamil Version)
Game Over (Telugu Version)
Gameboys Level-Up Edition
Garbage
Garfunkel and Oates: Trying to be Special
Garth Brooks: The Road I’m On
Garuda Di Dadaku
Gelo
Gemini
GEN HOSHINO STADIUM TOUR “POP VIRUS”
Genius
Gentlemen and Gangsters
George Lopez: We'll Do It For Half
Get Organized with The Home Edit
Getting Played
Ghost House
Ghost Stories
GHOUL
Gigi
Gina Yashere: Laughing to America
Giri

My Little Pony Equestria Girls: Legend of Everfree
My Little Pony Equestria Girls: Rainbow Rocks
My Little Pony Friendship Is Magic: Best Gift Ever
My Next Guest Needs No Introduction With David Letterman
My Next Guest with David Letterman and Shah Rukh Khan
My Only Mother
My Own Man
My Schoolmate, the Barbarian
My True Friend
My Way
Nadiya’s Time to Eat
Naga The Eternal Yogi
Naked
Naruto Shippûden the Movie: Bonds
Naruto Shippûden the Movie: The Will of Fire
Naruto Shippuden : Blood Prison
Naruto Shippuden: The Movie: The Lost Tower
Natsamrat - Asa Nat Hone Nahi
Natural Selection
Nature: Animals With Cameras
NATURE: Natural Born Hustlers
Nature: Raising the Dinosaur Giant
Nature's Great Events (2009)
Nature's Great Events: Diaries
New Money
Next
Night Moves
Nightmare Tenants, Slum Landlords
Nila
Nisman: The Prosecutor, the President, and the Spy
No Longer kids
No One Will Ever Know
Noblemen
Noddy Toyland Detective
Norm of the North: Keys to the Kingdom
Not Alone
Nothing to Lose
NOVA: 

The Incident
The Incredible Story of the Giant Pear
The Interview
The Island
The Karate Kid
The Killer
The Kite
The Land
The last hour
The Last Resort
The Legend of Cocaine Island
The Lift Boy
The Light of My Eyes
The Little Mermaid
The Little Vampire
The Long Goodbye: The Kara Tippetts Story
The Longest Yard
The Lovers
The lynx
The Mad Monk
The Magic School Bus Rides Again In the Zone
The Magic School Bus Rides Again Kids In Space
The Magic School Bus Rides Again The Frizz Connection
The Making of Frozen Planet
The Man Who Would Be Polka King
The Match
The Mayor
The Memphis Belle: A Story of a
Flying Fortress
The Midas Touch
The Missing Menu
The Mitfords: A Tale of Two Sisters
The Most Beautiful Hands of Delhi
The Mr. Peabody and Sherman Show
The Muppets
The Netflix Afterparty: The Best Shows of The Worst Year
The Old Thieves: The Legend of Artegios
The Only Mother To You All
The Other
The Other One: The Long Strange Trip of Bob Weir
The Outpost
The Outsider
The Perfect Day
The Perfec

In [59]:
netflix_data.head(5)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,IMDB_Rating
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...,"7.4 based on 22,005 user ratings"
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...,6.0 based on 574 user ratings
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow...",4.6 based on 853 user ratings
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi...",NaN
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...,"6.8 based on 230,888 user ratings"


In [57]:
#netflix_data.head(5)

netflix_data.show_id[1]

's2'

In [74]:
netflix_data.dropna()
netflix_data.reset_index(drop=True)

CleanNetflix = netflix_data
count = 0
for show in CleanNetflix.IMDB_Rating:
    if show == "NaN":
        CleanNetflix.drop([count], axis=0, inplace=True)
      
        count +=1
    else:
      
        count +=1
#netflix_data.to_csv("Netflix_dataset.csv", index=False)
print("Complete!")
print(count)

Complete!
7787


In [76]:
CleanNetflix.reset_index(drop=True)
CleanNetflix.head(25)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,IMDB_Rating
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...,"7.4 based on 22,005 user ratings"
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...,6.0 based on 574 user ratings
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow...",4.6 based on 853 user ratings
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...,"6.8 based on 230,888 user ratings"
5,s6,TV Show,46,Serdar Akar,"Erdal Beşikçioğlu, Yasemin Allen, Melis Birkan...",Turkey,"July 1, 2017",2016,TV-MA,1 Season,"International TV Shows, TV Dramas, TV Mysteries",A genetics professor experiments with a treatm...,7.2 based on 12 user ratings
6,s7,Movie,122,Yasir Al Yasiri,"Amina Khalil, Ahmed Dawood, Tarek Lotfy, Ahmed...",Egypt,"June 1, 2020",2019,TV-MA,95 min,"Horror Movies, International Movies","After an awful accident, a couple admitted to ...","6.8 based on 3,893 user ratings"
7,s8,Movie,187,Kevin Reynolds,"Samuel L. Jackson, John Heard, Kelly Rowan, Cl...",United States,"November 1, 2019",1997,R,119 min,Dramas,After one of his high school students attacks ...,"6.7 based on 22,263 user ratings"
8,s9,Movie,706,Shravan Kumar,"Divya Dutta, Atul Kulkarni, Mohan Agashe, Anup...",India,"April 1, 2019",2019,TV-14,118 min,"Horror Movies, International Movies","When a doctor goes missing, his psychiatrist w...",5.3 based on 417 user ratings
9,s10,Movie,1920,Vikram Bhatt,"Rajneesh Duggal, Adah Sharma, Indraneil Sengup...",India,"December 15, 2017",2008,TV-MA,143 min,"Horror Movies, International Movies, Thrillers",An architect and his wife move into a castle t...,"6.4 based on 2,978 user ratings"
10,s11,Movie,1922,Zak Hilditch,"Thomas Jane, Molly Parker, Dylan Schmid, Kaitl...",United States,"October 20, 2017",2017,TV-MA,103 min,"Dramas, Thrillers",A farmer pens a confession admitting to his wi...,"6.3 based on 54,101 user ratings"


In [78]:
CleanNetflix.to_csv("CLeanNetflixData.csv", index=False)